In [ ]:
%cd ~/dojo-pytest/

# Dojo Pytest (Ou comment faire des tests qui roxxent du poney)

Objectif : faire découvrir de manière peut-être un petit peu plus avancée le framework de test que nous utilisons.

Sujet dense : ce ne sera pas exhaustif.

(Et RISE, c'est bien aussi)

RISE: Outil utilisé pour cette présentation. Basé sur Reveal.JS
    
https://rise.readthedocs.io/en/latest/

# Avant de commencer :

Versions utilisées

In [238]:
%%bash
python --version
pytest --version

Python 3.10.12
pytest 7.4.4


<a id="bases"></a>
# Bases

# Exemple simple de test :

In [240]:
%%writefile tests/bases/test_simple_example.py
#!/usr/bin/env python3

def test_foo():
    assert True

Overwriting tests/bases/test_simple_example.py


`%%writefile` -> commande magique (feature de IPython utilisable dans les notebooks Jupyter).

Permet de faire autre-chose que de l'exécution de python par défaut.
Autres exemples :
- `%cd`
- `%%script`
.
.
.

https://ipython.readthedocs.io/en/stable/interactive/magics.html


Par défaut :
- Infos utiles par défaut dans l'affichage
- récapitulatif des tests + leurs résultats.


`%%bash` -> Autre commande magique IPython.

In [241]:
%%bash
pytest tests/bases/test_simple_example.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/bases/test_simple_example.py .                                     [100%]

============================== 1 passed in 0.01s ===============================


# Résultats possibles
Il y en a principalement 3 :
- Success.
- Failure.
- Error.

Autres retours possibles :
- `xfail`
- `xpass`
- `skip`

Relativement anecdotiques.

#### Dans le cas d'un succès (Success) :
Pas d'exception non gérée, tout s'est bien passé.

In [242]:
%%writefile tests/bases/test_results_example_success.py
#!/usr/bin/env python3

def test_success():
    pass

Overwriting tests/bases/test_results_example_success.py


In [243]:
%%script zsh --no-raise-error
pytest tests/bases/test_results_example_success.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/bases/test_results_example_success.py .                            [100%]

============================== 1 passed in 0.01s ===============================


À partir de maintenant, plus la magic command `%%bash`.
(Retourne une erreur si la commande a un statut de retour autre que 0).

`%%script bash --no-raise-error` permet d'éviter ce problème.

#### Dans le cas d'un échec (Failure) :
Une exception (de n'importe quel type) a été levée sans être gérée: le test n'est pas passé.

Différence entre failure et error : source https://stackoverflow.com/a/32103555/3156085

In [244]:
%%writefile tests/bases/test_results_example_failure.py
#!/usr/bin/env python3

def test_failure():
    assert False, "This test can only fail."

def test_failure_alt():
    raise AssertionError("This test can only fail.")
     
def test_failure_other_exc():
    raise ValueError("A test can fail with something else than an AssertionError.")
    
def test_failure_with_division_error():
    a = 1/0

Overwriting tests/bases/test_results_example_failure.py


In [245]:
%%script bash --no-raise-error
pytest -v tests/bases/test_results_example_failure.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 4 items

tests/bases/test_results_example_failure.py::test_failure FAILED         [ 25%]
tests/bases/test_results_example_failure.py::test_failure_alt FAILED     [ 50%]
tests/bases/test_results_example_failure.py::test_failure_other_exc FAILED [ 75%]
tests/bases/test_results_example_failure.py::test_failure_with_division_error FAILED [100%]

=================================== FAILURES ===================================
_________________________________ test_failure _________________________________

    def test_failure():
>       assert False, "This test can only fail."
E       AssertionError: This test can only fail.
E       assert False

tests/bases/test_results_example_failure.py:4: Asserti

#### Dans le cas d'une erreur (Error)
Un problème est survenu _avant_ que le test n'ait été lancé (dans le setup).

In [246]:
%%writefile tests/bases/test_results_example_error.py
#!/usr/bin/env python3
import pytest

@pytest.fixture
def foo():
    return 1/0

def test_error(foo):
    """This will issue an error because of ZeroDivisionError in fixture"""
    pass

Overwriting tests/bases/test_results_example_error.py


In [247]:
%%script bash --no-raise-error
pytest tests/bases/test_results_example_error.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/bases/test_results_example_error.py E                              [100%]

==================================== ERRORS ====================================
_________________________ ERROR at setup of test_error _________________________

    @pytest.fixture
    def foo():
>       return 1/0
E       ZeroDivisionError: division by zero

tests/bases/test_results_example_error.py:6: ZeroDivisionError
=========================== short test summary info ============================
ERROR tests/bases/test_results_example_error.py::test_error - ZeroDivisionError: division by zero
=============================== 1 error in 0.10s ===============================


# Et si on attend la levée d'une exception ?

On peut dans ce cas utiliser le manager de contexte [`pytest.raises`](https://docs.pytest.org/en/4.6.x/reference.html#pytest-raises) prévu spécialement pour ce genre de cas.

Il prend en paramètre le type d'exception attendue.

In [249]:
%%writefile tests/bases/test_raises.py
#!/usr/bin/env python3
import pytest

def f_to_test():
    d = {"foo": 42}
    a =  This will raise a KeyError
    
def test_f_to_test():
    with pytest.raises(KeyError):
        f_to_test()

def test_f_to_test_bis():
    """This test should fail."""
    with pytest.raises(ValueError):
        f_to_test()
        
def test_without_exception():
    """This test should also fail."""
    with pytest.raises(ValueError):
        pass

Overwriting tests/bases/test_raises.py


In [250]:
%%script bash --no-raise-error
pytest -v tests/bases/test_raises.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 0 items / 1 error

==================================== ERRORS ====================================
_________________ ERROR collecting tests/bases/test_raises.py __________________
../.envs/dojo-pytest/lib/python3.10/site-packages/_pytest/python.py:617: in _importtestmodule
    mod = import_path(self.path, mode=importmode, root=self.config.rootpath)
../.envs/dojo-pytest/lib/python3.10/site-packages/_pytest/pathlib.py:567: in import_path
    importlib.import_module(module_name)
/usr/lib/python3.10/importlib/__init__.py:126: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
<frozen importlib._bootstrap>:1050: in _gcd_import
    ???
<frozen importlib._bootstrap>:1027: 

# Conventions de nommage

Pour qu'un test soit collecté par le comportement par défaut de pytest :
- Il doit se trouver dans le package courant ou un de ses sous-packages.
- Le module dans lequel il se trouve doit avoir son nom préfixé par "test_" (ou suffixé par "_test.py")
- Le nom du test doit être préfixé par "test".
- S'il se trouve dans une classe de test (sous forme de méthode), la classe doit avoir un nom préfixé par "Test" et la classe en question ne doit pas avoir de méthode `__init__()`.


Source: https://docs.pytest.org/en/7.1.x/explanation/goodpractices.html

Tout ça reste aussi des comportements qui peuvent être modifiés par configuration.

# Exemple :

Le paramètre `--collect-only` sert à se contenter de collecter les tests. C'est-à-dire lister ceux qui sont disponibles.

`pygmentize` est un équivalent de `cat` mais avec la coloration syntaxique de Python.
https://pygments.org/docs/quickstart/

Voici l'exemple d'une utilisation de pytest où on se contente de passer un dossier plutôt qu'un fichier spécifique. Il s'occupera alors de collecter tout seul les tests disponibles dans l'arborescence.

Il y a aussi la possibilité de préciser un test particulier au sein d'un fichier de test à l'aide de l'opérateur `::`.

In [252]:
%%bash
set -x
tree -I __pycache__ tests/bases/naming_conventions/
pygmentize tests/bases/naming_conventions/test_collected_tests.py

+ tree -I __pycache__ tests/bases/naming_conventions/


tests/bases/naming_conventions/
├── not_collected_tests.py
└── test_collected_tests.py

0 directories, 2 files


+ pygmentize tests/bases/naming_conventions/test_collected_tests.py


def basic_test():
    pass


def test_basic():
    pass


class TestMyTestclass:
    def basic_test(self):
        pass

    def test_basic(self):
        pass


class MyTestclass:
    def basic_test(self):
        pass

    def test_basic(self):
        pass


class TestMyTestclassWithInit:
    def __init__(self):
        pass

    def basic_test(self):
        pass

    def test_basic(self):
        pass


In [253]:
%%script bash --no-raise-error
pytest --collect-only tests/bases/naming_conventions

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 2 items

<Module tests/bases/naming_conventions/test_collected_tests.py>
  <Function test_basic>
  <Class TestMyTestclass>
    <Function test_basic>

=============================== warnings summary ===============================
tests/bases/naming_conventions/test_collected_tests.py:25
  /home/vmonteco/dojo-pytest/tests/bases/naming_conventions/test_collected_tests.py:25: PytestCollectionWarning: cannot collect test class 'TestMyTestclassWithInit' because it has a __init__ constructor (from: tests/bases/naming_conventions/test_collected_tests.py)
    class TestMyTestclassWithInit:

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================== 2 tests collected in 0.01s ==========================


# Exemple plus complet

Voici un exemple plus complet qui met en évidence les différences avec et sans `__init__.py`. (named packages vs. package IIRC)

In [ ]:
%%bash
set -x
tree -I __pycache__ tests/bases/naming_conventions_more_complete
pygmentize tests/bases/naming_conventions_more_complete/subdir1/test_collected_tests.py

In [ ]:
%%bash
pytest --collect-only tests/bases/naming_conventions_more_complete

### Note :
Sans les `__init__.py` il semblerait que l'on puisse avoir quelque-chose qui ressemble à des erreurs d'import (collisions?).

Comme [ici](https://stackoverflow.com/questions/53918088/import-file-mismatch-in-pytest).


# Concepts

## Paramétrisation
On veut parfois faire un même test pour plusieurs cas différents.
Afin d'éviter la duplication, on peut simplement utiliser le même test avec plusieurs paramètres en entrée.

Cela se déclare avec le décorateur [`pytest.mark.parametrize`](https://docs.pytest.org/en/6.2.x/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions).

### Exemple :

In [222]:
%%writefile tests/concepts/parametrization/simple_parametrization/test_simple_parametrization_example.py
#!/usr/bin/env python3
import pytest

@pytest.mark.parametrize("mybool", [False, True])
def test_simple_parametrization_example(mybool):
    print("\nValue of mybool: %r" % mybool)

Overwriting tests/concepts/parametrization/simple_parametrization/test_simple_parametrization_example.py


In [ ]:
%%script bash --no-raise-error
pytest -s tests/concepts/parametrization/simple_parametrization/test_simple_parametrization_example.py

### Exemples avec un cas simple :
#### Sans paramétrisation

In [255]:
%%writefile tests/concepts/parametrization/example_with_simple_case/without_parametrization.py
#!/usr/bin/env python3

def is_even(i):
    return (i % 2) == 0

def test_is_even_1():
    assert not is_even(1)

def test_is_even_2():
    assert is_even(2) 

Overwriting tests/concepts/parametrization/example_with_simple_case/without_parametrization.py


In [256]:
%%script bash --no-raise-error
pytest -v tests/concepts/parametrization/example_with_simple_case/without_parametrization.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items

tests/concepts/parametrization/example_with_simple_case/without_parametrization.py::test_is_even_1 PASSED [ 50%]
tests/concepts/parametrization/example_with_simple_case/without_parametrization.py::test_is_even_2 PASSED [100%]

============================== 2 passed in 0.01s ===============================


### Exemples avec un cas simple :
#### Avec paramétrisation :

In [258]:
%%writefile tests/concepts/parametrization/example_with_simple_case/with_parametrization.py
#!/usr/bin/env python3
import pytest

def is_even(i):
    return (i % 2) == 0

@pytest.mark.parametrize("number,parity", [(0, True) , (1, False), (2, True)])
def test_is_event(number, parity):
    assert is_even(number) == parity

Overwriting tests/concepts/parametrization/example_with_simple_case/with_parametrization.py


In [259]:
%%script bash --no-raise-error
pytest -v tests/concepts/parametrization/example_with_simple_case/with_parametrization.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 3 items

tests/concepts/parametrization/example_with_simple_case/with_parametrization.py::test_is_event[0-True] PASSED [ 33%]
tests/concepts/parametrization/example_with_simple_case/with_parametrization.py::test_is_event[1-False] PASSED [ 66%]
tests/concepts/parametrization/example_with_simple_case/with_parametrization.py::test_is_event[2-True] PASSED [100%]

============================== 3 passed in 0.02s ===============================


### Truc : Chaînage de décorateurs
Parfois, la paramétrisation peut être lourde à rédiger si les fonctions à tester ont beaucoup d'entrées possibles.

La combinatoire peut donner beaucoup de cas !

Chaîner les décorateurs parametrize peut donc considérablement alléger leur écriture.

#### Exemples :
##### Sans le chaînage :

In [260]:
%%writefile tests/concepts/parametrization/chaining_trick/without_chaining.py
#!/usr/bin/env python3
import pytest

@pytest.mark.parametrize("bool1,bool2,bool3,bool4", [
    (False, False, False, False),
    (False, False, False, True),
    (False, False, True, False),
    (False, False, True, True),
    (False, True, False, False),
    (False, True, False, True),
    (False, True, True, False),
    (False, True, True, True),
    (True, False, False, False),
    (True, False, False, True),
    (True, False, True, False),
    (True, False, True, True),
    (True, True, False, False),
    (True, True, False, True),
    (True, True, True, False),
    (True, True, True, True),
])
def test_complicated_function_with_many_inputs(bool1, bool2, bool3, bool4):
    print("\nbool1: %r - bool2: %r - bool3: %r - bool4: %r" % (bool1, bool2, bool3, bool4))

Overwriting tests/concepts/parametrization/chaining_trick/without_chaining.py


In [261]:
%%script bash --no-raise-error
pytest -v tests/concepts/parametrization/chaining_trick/without_chaining.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 16 items

tests/concepts/parametrization/chaining_trick/without_chaining.py::test_complicated_function_with_many_inputs[False-False-False-False] PASSED [  6%]
tests/concepts/parametrization/chaining_trick/without_chaining.py::test_complicated_function_with_many_inputs[False-False-False-True] PASSED [ 12%]
tests/concepts/parametrization/chaining_trick/without_chaining.py::test_complicated_function_with_many_inputs[False-False-True-False] PASSED [ 18%]
tests/concepts/parametrization/chaining_trick/without_chaining.py::test_complicated_function_with_many_inputs[False-False-True-True] PASSED [ 25%]
tests/concepts/parametrization/chaining_trick/without_chaining.py::test_complicated_function_with_many_

#### Examples :
##### Avec le chaînage :

In [264]:
 %%writefile tests/concepts/parametrization/chaining_trick/with_chaining.py
#!/usr/bin/env python3
import pytest

@pytest.mark.parametrize("bool1", (False, True))
@pytest.mark.parametrize("bool2", (False, True))
@pytest.mark.parametrize("bool3", (False, True))
@pytest.mark.parametrize("bool4", (False, True))
def test_complicated_function_with_many_inputs(bool1, bool2, bool3, bool4):
    print("\nbool1: %r - bool2: %r - bool3: %r - bool4: %r" % (bool1, bool2, bool3, bool4))

Overwriting tests/concepts/parametrization/chaining_trick/with_chaining.py


In [263]:
%%script bash --no-raise-error
pytest -v tests/concepts/parametrization/chaining_trick/with_chaining.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 16 items

tests/concepts/parametrization/chaining_trick/with_chaining.py::test_complicated_function_with_many_inputs[False-False-False-False] PASSED [  6%]
tests/concepts/parametrization/chaining_trick/with_chaining.py::test_complicated_function_with_many_inputs[False-False-False-True] PASSED [ 12%]
tests/concepts/parametrization/chaining_trick/with_chaining.py::test_complicated_function_with_many_inputs[False-False-True-False] PASSED [ 18%]
tests/concepts/parametrization/chaining_trick/with_chaining.py::test_complicated_function_with_many_inputs[False-False-True-True] PASSED [ 25%]
tests/concepts/parametrization/chaining_trick/with_chaining.py::test_complicated_function_with_many_inputs[False-Tr

## Fixtures

L'une des fonctionalités clés de pytest est l'usage de fixtures. Les fixtures sont des fonctions décorées par [`@pytest.fixture`](https://docs.pytest.org/en/7.1.x/reference/reference.html#pytest-fixture-api). Elles permettent de préparer les tests, par exemple avec des sets de données.

## Exemple et usage :

In [266]:
%%writefile tests/concepts/fixtures/simple_example.py
#!/usr/bin/env python3
import pytest

@pytest.fixture
def myfixture():
      return "FOO"

def test_fixture(myfixture):
    print("\n%s\n" % myfixture)

Overwriting tests/concepts/fixtures/simple_example.py


In [267]:
%%script bash --no-raise-error
pytest tests/concepts/fixtures/simple_example.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/fixtures/simple_example.py .                              [100%]

============================== 1 passed in 0.01s ===============================


## Une fixture peut faire appel à une autre fixture :

In [268]:
%%writefile tests/concepts/fixtures/fixture_as_fixture_dependency.py
#!/usr/bin/env python3
import pytest

@pytest.fixture
def my_first_fixture():
    return "FOO"

@pytest.fixture
def my_second_fixture(my_first_fixture):
    return my_first_fixture * 2

def test_fixture_as_fixture_dependency(my_second_fixture):
    print("\n%s\n" % my_second_fixture)

Overwriting tests/concepts/fixtures/fixture_as_fixture_dependency.py


In [269]:
%%script bash --no-raise-error
pytest -s tests/concepts/fixtures/fixture_as_fixture_dependency.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/fixtures/fixture_as_fixture_dependency.py 
FOOFOO

.

============================== 1 passed in 0.01s ===============================


## Paramétrisation de fixture :

Il est possible, tout comme pour les tests eux-mêmes, de paramétriser des tests.

In [272]:
%%writefile tests/concepts/fixtures/fixtures_parametrisation.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(params=[True, False])
def parametrized_fixture(request):
    return request.param

def test_fixture_parametrization(parametrized_fixture):
    print("\n%r\n" % parametrized_fixture)

Overwriting tests/concepts/fixtures/fixtures_parametrisation.py


In [271]:
%%script bash --no-raise-error
pytest -s tests/concepts/fixtures/fixtures_parametrisation.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 2 items

tests/concepts/fixtures/fixtures_parametrisation.py 
True

.
False

.

============================== 2 passed in 0.01s ===============================


### Problème possible : vouloir utiliser une fixture plusieurs fois avec paramétrisation pour un même test

Quid si on a deux fixtures différentes qui veulent faire appel à une troisième, mais sans que l'on souhaite avoir les mêmes valeurs (dupliquer une même fixture)

Le problème rencontré est alors que la fixture "racine" n'est pas dupliquée (la même valeur est utilisée à chaque fois et on ne pourrait pas par exemple pour une valeur booléenne faire cohabiter un `False` et un `True`.

In [ ]:
%%writefile tests/concepts/fixtures/fixtures_parametrisation_test_duplicate_direct.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(params=[True, False])
def boolean(request):
    return request.param

# This doesn't use fixture as fixtures but as local decorated functions.
def test_duplicate_fixture(bool1=boolean, bool2=boolean):
    print("%r - %r\n" % (bool1, bool2))

In [ ]:
%%script bash --no-raise-error
pytest -s tests/concepts/fixtures/fixtures_parametrisation_test_duplicate_direct.py

In [ ]:
%%writefile tests/concepts/fixtures/fixtures_parametrisation_test_duplicate.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(params=[True, False])
def boolean(request):
    return request.param

@pytest.fixture
def bool1(boolean):
    return boolean

@pytest.fixture
def bool2(boolean):
    return boolean

# With this, each fixture (bool1, bool2) depends on the same fixture boolean. Only two cases.
def test_duplicate_fixture(bool1, bool2):
    print("%r - %r\n" % (bool1, bool2))

In [ ]:
%%script bash --no-raise-error
pytest --collect-only tests/concepts/fixtures/fixtures_parametrisation_test_duplicate.py

## Workaround

In [ ]:
%%writefile tests/concepts/fixtures/fixtures_parametrisation_test_workaround.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(params=[True, False])
def boolean(request):
    return request.param

# You have to make a copy of the fixture with a new name.
boolean_bis = boolean

def test_duplicate_fixture(boolean, boolean_bis):
    print("\n%r - %r\n" % (boolean, boolean_bis))

In [ ]:
%%script bash --no-raise-error
pytest -s tests/concepts/fixtures/fixtures_parametrisation_test_workaround.py

## NOTE :
Question : peut-être devrait-on mettre au sein de la suite de test qui représenteraient des set de valeurs différentes (Utilisateurs avec droits avancés, booléens, Enums...)

## Conftest :

Les fichiers conftest.py sont simplement les fichiers de configuration des suites de test.

Typiquement les fichiers dans lesquels seront stockées les fixtures.

Les fixtures seront accessibles par tous les tests (Et fixtures) dans l'arborescence du dossier dans lequel le `contest.py` se trouve.

### Exemple :

In [273]:
%%writefile tests/concepts/fixtures/conftest_example/conftest.py
#!/usr/bin/env python3
import pytest

@pytest.fixture
def myfixture():
    return __file__

Overwriting tests/concepts/fixtures/conftest_example/conftest.py


In [281]:
%%writefile tests/concepts/fixtures/conftest_example/test_mytest.py
#!/usr/bin/env python3

def test_mytest(myfixture):
    print("\n%s" % myfixture)

Overwriting tests/concepts/fixtures/conftest_example/test_mytest.py


In [283]:
%%script bash --no-raise-error
mkdir tests/concepts/fixtures/conftest_example
mkdir tests/concepts/fixtures/conftest_example/subdir1
mkdir tests/concepts/fixtures/conftest_example/subdir2
touch tests/concepts/fixtures/conftest_example/{__init__.py,subdir1/__init__.py,subdir2/__init__.py}
cp tests/concepts/fixtures/conftest_example/{conftest.py,subdir1/}
cp tests/concepts/fixtures/conftest_example/{test_mytest.py,subdir1/}
cp tests/concepts/fixtures/conftest_example/{test_mytest.py,subdir2/}
set -x
tree -I __pycache__ tests/concepts/fixtures/conftest_example
pytest -sv tests/concepts/fixtures/conftest_example

mkdir: cannot create directory ‘tests/concepts/fixtures/conftest_example’: File exists
mkdir: cannot create directory ‘tests/concepts/fixtures/conftest_example/subdir1’: File exists
mkdir: cannot create directory ‘tests/concepts/fixtures/conftest_example/subdir2’: File exists
+ tree -I __pycache__ tests/concepts/fixtures/conftest_example


tests/concepts/fixtures/conftest_example
├── __init__.py
├── conftest.py
├── subdir1
│   ├── __init__.py
│   ├── conftest.py
│   └── test_mytest.py
├── subdir2
│   ├── __init__.py
│   └── test_mytest.py
└── test_mytest.py

2 directories, 8 files


+ pytest -sv tests/concepts/fixtures/conftest_example


============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 3 items

tests/concepts/fixtures/conftest_example/test_mytest.py::test_mytest 
/home/vmonteco/dojo-pytest/tests/concepts/fixtures/conftest_example/conftest.py
PASSED
tests/concepts/fixtures/conftest_example/subdir1/test_mytest.py::test_mytest 
/home/vmonteco/dojo-pytest/tests/concepts/fixtures/conftest_example/subdir1/conftest.py
PASSED
tests/concepts/fixtures/conftest_example/subdir2/test_mytest.py::test_mytest 
/home/vmonteco/dojo-pytest/tests/concepts/fixtures/conftest_example/conftest.py
PASSED

============================== 3 passed in 0.04s ===============================


## Scope :
La déclaration de fixture peut prendre en paramètre un paramètre `scope` inclus dans 5 valeurs :
- `"function"` (valeur par défaut)
- `"class"`
- `"module"`
- `"package"`
- `"session"`

Ce paramètre permet de réutiliser la même fixture au sein de plusieurs tests.

Ainsi, une fixture avec le scope `"session"` sera invoquée une fois par session de test, une fois par package avec le scope `"package"`, etc.

L'intérêt peut se trouver dans la recherche de meilleures performances. Avec des fixtures coûteuses en ressources dont l'invocation sera gérée plus finement, avec une mise en cache le reste du temps.

In [286]:
%%writefile tests/concepts/fixtures/scope/package1/conftest.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(scope="function")
def function_fixture():
    print("Function fixture called.")
    
@pytest.fixture(scope="class")
def class_fixture():
    print("Class fixture called.")

@pytest.fixture(scope="module") 
def module_fixture():
    print("Module fixture called.")

@pytest.fixture(scope="package")
def package_fixture():
    print("Package fixture called.")

Overwriting tests/concepts/fixtures/scope/package1/conftest.py


In [287]:
%%writefile tests/concepts/fixtures/scope/conftest.py
#!/usr/bin/env python3
import pytest

@pytest.fixture(scope="session")
def session_fixture():
    print("Session fixture called.")

Overwriting tests/concepts/fixtures/scope/conftest.py


In [291]:
%%writefile tests/concepts/fixtures/scope/package1/test_scope.py
#!/usr/bin/env python3

class TestClass1:
    def test_scope_1(self, function_fixture, class_fixture, module_fixture, package_fixture, session_fixture):
        pass
    
    def test_scope_2(self, function_fixture, class_fixture, module_fixture, package_fixture, session_fixture):
        pass

class TestClass2:
    def test_scope_1(self, function_fixture, class_fixture, module_fixture, package_fixture, session_fixture):
        pass

    def test_scope_2(self, function_fixture, class_fixture, module_fixture, package_fixture, session_fixture):
        pass

Overwriting tests/concepts/fixtures/scope/package1/test_scope.py


In [290]:
%%script bash --no-raise-error
mkdir tests/concepts/fixtures/scope/package2
cp tests/concepts/fixtures/scope/{package1/test_scope.py,package2/test_scope.py}
cp tests/concepts/fixtures/scope/{package1/conftest.py,package2/conftest.py}
touch tests/concepts/fixtures/scope/{__init__.py,package1/__init__.py,package2/__init__.py}

pytest -s tests/concepts/fixtures/scope

mkdir: cannot create directory ‘tests/concepts/fixtures/scope/package2’: File exists


============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 8 items

tests/concepts/fixtures/scope/package1/test_scope.py Session fixture called.
Package fixture called.
Module fixture called.
Class fixture called.
Function fixture called.
.Function fixture called.
.Class fixture called.
Function fixture called.
.Function fixture called.
.
tests/concepts/fixtures/scope/package2/test_scope.py Package fixture called.
Module fixture called.
Class fixture called.
Function fixture called.
.Function fixture called.
.Class fixture called.
Function fixture called.
.Function fixture called.
.

============================== 8 passed in 0.05s ===============================


## Mocking :

### Principe :

Le mocking, dans le monde des test est le remplacement d'un élément par un autre avec des fonctionnalités utiles aux tests.

C'est une pratique qui peut représenter plusieurs intérêts, parmi lesquelles :
- Gain de performances pour l'exécution des tests.
- Separation of concerns.
- Introspection sur le comportement du code à des fins de test.

## Gain de performances pour l'exécution des tests :
    Si une fonctionnalité à tester repose sur une méthode impliquant un calcul lourd et long, on pourra gagner du temps en mockant un objet renvoyant instantannément la valeur.

## Separation of concerns :
    On va sans doute vouloir aussi isoler au maximum le comportement de la feature testée. Si je veux tester une fonctionnalité A qui dépend de fonctionnalités B et C, le rôle de mon test sera de tester la fonctionnalité A sans que le résultat ne dépende des fonctionnalités B et C (On peut supposer que celles-ci sont cassées, pas encore codées...).

## Introspection sur le comportement du code à des fins de test :
    Cela peut être utile de s'assurer qu'une fonction a bien été appelée, avec tels paramètres...

### `pytest-mock`/`unittest.mock` :

`pytest-mock` est un paquet mettant à disposition de pytest un wrapper de la librairie `unittest.mock` de la librairie standard.

Ce wrapper est accessible par la fixture `mocker` (qui la même api que `unittest.mock.patch`).

#### Mocker une simple constante avec `mocker.path.object` :

Voici un exemple simple pour commencer, on peut se contenter de mocker quelque-chose d'aussi simple qu'une constante.

In [293]:
%%writefile tests/concepts/mocking/pytest-mock/simple_constant_mock/check_pwd_length.py
#!/usr/bin/env python3
PERMIT_SHORT_PASSWORDS = False

def check_pwd_length(pwd):
    print(f"{PERMIT_SHORT_PASSWORDS!r}")
    if PERMIT_SHORT_PASSWORDS:
        assert len(pwd) >= 8
    else:
        assert len(pwd) >= 16

Overwriting tests/concepts/mocking/pytest-mock/simple_constant_mock/check_pwd_length.py


In [294]:
%%writefile tests/concepts/mocking/pytest-mock/simple_constant_mock/test_simple_constant_mock.py
import check_pwd_length

def test_check_pwd_length_for_short_passwords(mocker):
    mocker.patch.object(check_pwd_length, "PERMIT_SHORT_PASSWORDS", True)
    check_pwd_length.check_pwd_length("petitpatapon")

Overwriting tests/concepts/mocking/pytest-mock/simple_constant_mock/test_simple_constant_mock.py


[`mocker.patch.object`](https://docs.python.org/3.10/library/unittest.mock.html#patch-object) est ici utilisé avec 3 paramètres :
- L'objet à patcher.
- Le nom de l'attribut à patcher.
- L'objet avec lequel remplacer l'attribut.

In [295]:
%%script bash --no-raise-error
pytest -s tests/concepts/mocking/pytest-mock/simple_constant_mock/test_simple_constant_mock.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/mocking/pytest-mock/simple_constant_mock/test_simple_constant_mock.py True
.

============================== 1 passed in 0.01s ===============================


#### Note: On peut aussi utiliser `mocker.patch.object` avec deux arguments :

On peut omettre l'objet avec lequel patcher la cible. Un mock en sera alors fait.

In [296]:
%%writefile tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/check_pwd_length.py
#!/usr/bin/env python3
PERMIT_SHORT_PASSWORDS = False


def check_pwd_length(pwd):
    print(f"{PERMIT_SHORT_PASSWORDS!r}")
    if PERMIT_SHORT_PASSWORDS:
        assert len(pwd) >= 8
    else:
        assert len(pwd) >= 16

Overwriting tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/check_pwd_length.py


In [299]:
%%writefile tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/test_simple_constant_mock.py
import check_pwd_length

def test_check_pwd_length_for_short_passwords(mocker):
    mock = mocker.patch.object(check_pwd_length, "PERMIT_SHORT_PASSWORDS")
    check_pwd_length.check_pwd_length("petitpataponpluslong")
    mock.assert_not_called()


Overwriting tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/test_simple_constant_mock.py


In [298]:
%%script bash --no-raise-error
pytest -s tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/test_simple_constant_mock.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/mocking/pytest-mock/mocker_patch_object_two_params/test_simple_constant_mock.py <MagicMock name='PERMIT_SHORT_PASSWORDS' id='140388908180784'>
.

============================== 1 passed in 0.01s ===============================


In [ ]:
#!/usr/bin/env python3
"""
Attempt to merge both file from previous snippet.
But requires self-referencing the current module as target.
PEP 3130 would've allowed that but was rejected.
https://peps.python.org/pep-3130/
"""
PERMIT_SHORT_PASSWORDS = False

def check_pwd_length(pwd):
    if PERMIT_SHORT_PASSWORDS:
        assert len(pwd) >= 8
    else:
        assert len(pwd) >= 16

def test_check_pwd_length_for_short_passwords(mocker):
    with mocker.patch.object(__module__, "PERMIT_SHORT_PASSWORDS", True):
        check_pwd_length("petitpatapon")

Un peu limité avec une constante. :)

Les methodes des objets `Mock` liées à l'examen de l'utilisation des fonctions n'a pas vraiment de sens ici.
Mais il existe des classes de mock dédiées aux non-callables.

### Mocker une fonction avec `mocker.patch` :

In [301]:
%%writefile tests/concepts/mocking/pytest-mock/patching_a_function/answer.py
#!/usr/bin/env python3
from time import sleep

def very_long_and_faulty_calculus():
    sleep(5)
    raise Exception("Oops")
    return 42

def get_answer_to_everything():
    return very_long_and_faulty_calculus()

Overwriting tests/concepts/mocking/pytest-mock/patching_a_function/answer.py


In [304]:
%%writefile tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py
#!/usr/bin/env python3
from answer import get_answer_to_everything

def test_get_answer_to_everything_the_long_way():
    assert get_answer_to_everything() == 42
    
def test_get_answer_to_everything(mocker):
    mock = mocker.patch("answer.very_long_and_faulty_calculus", return_value=42)
    assert get_answer_to_everything() == 42
    mock.assert_called()


Overwriting tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py


In [305]:
%%script bash --no-raise-error
pytest -sv tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items

tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py::test_get_answer_to_everything_the_long_way FAILED
tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py::test_get_answer_to_everything PASSED

=================================== FAILURES ===================================
__________________ test_get_answer_to_everything_the_long_way __________________

    def test_get_answer_to_everything_the_long_way():
>       assert get_answer_to_everything() == 42

tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
tests/concepts/mocking/pytest-mock/patching_a

### Utilisations en tant que décorateurs :



In [306]:
%%writefile tests/concepts/mocking/pytest-mock/use_as_decorators/answer.py
#!/usr/bin/env python3
from time import sleep

def calculus():
    return 21

def get_answer_to_everything():
    return calculus()

Overwriting tests/concepts/mocking/pytest-mock/use_as_decorators/answer.py


In [307]:
%%writefile tests/concepts/mocking/pytest-mock/use_as_decorators/test_answers.py
#!/usr/bin/env python3
from answer import get_answer_to_everything
from unittest.mock import patch
import answer

@patch("answer.calculus", lambda: 42)
def test_in_function():
    assert get_answer_to_everything() == 42

@patch.object(answer, "calculus", return_value=42)
def test_in_function_bis(mock_method):
    assert get_answer_to_everything() == 42
    mock_method.assert_called_once()
    
@patch("answer.calculus", lambda: 42)
class TestInClass:
    def test_answer(self):
        assert get_answer_to_everything() == 42

@patch.object(answer, "calculus", return_value=42)
class TestInClassBis:
    def test_answer(self, mock_method):
        assert get_answer_to_everything() == 42
        mock_method.assert_called_once()

Overwriting tests/concepts/mocking/pytest-mock/use_as_decorators/test_answers.py


In [309]:
%%script bash --no-raise-error
pytest -s tests/concepts/mocking/pytest-mock/use_as_decorators/test_answers.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 4 items

tests/concepts/mocking/pytest-mock/use_as_decorators/test_answers.py ....

============================== 4 passed in 0.01s ===============================


### Les méthodes des objets Mock :

Lors des tests, les objets de type `Mock` créés par les patchers mettent à disposition un snemble de méthodes afin de faire les tests.

Les noms sont assez explicites.

In [311]:
%%writefile tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py
#!/usr/bin/env python3
from answer import get_answer_to_everything


def test_get_answer_to_everything(mocker):
    mock = mocker.patch("answer.very_long_and_faulty_calculus", return_value=42)
    mock.assert_not_called()
    assert get_answer_to_everything() == 42
    mock.assert_called_once()
    mock.return_value = 43
    assert get_answer_to_everything() == 43
    mock.assert_called()
    assert mock.return_value == 43
    assert mock.call_count == 2


Overwriting tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py


In [313]:
%%script bash --no-raise-error
pytest -sv tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 1 item

tests/concepts/mocking/pytest-mock/patching_a_function/test_answer.py::test_get_answer_to_everything PASSED

============================== 1 passed in 0.01s ===============================


### Monkeypatch

[`monkeypatch`](https://docs.pytest.org/en/6.2.x/monkeypatch.html) est une fonctionnalité par défaut de pytest mise à disposition par la fixture built-in du même nom. Elle permet via un certain nombre de méthodes de gérer également les patchs dans les tests.

In [314]:
%%writefile tests/concepts/mocking/monkeypatch/simple_constant_mock/check_pwd_length.py
#!/usr/bin/env python3
PERMIT_SHORT_PASSWORDS = False

def check_pwd_length(pwd):
    print(f"{PERMIT_SHORT_PASSWORDS!r}")
    if PERMIT_SHORT_PASSWORDS:
        assert len(pwd) >= 8
    else:
        assert len(pwd) >= 16

Overwriting tests/concepts/mocking/monkeypatch/simple_constant_mock/check_pwd_length.py


In [315]:
%%writefile tests/concepts/mocking/monkeypatch/simple_constant_mock/test_simple_constant_mock.py
#!/usr/bin/env python3
import check_pwd_length

def test_check_pwd_length_for_short_passwords(monkeypatch):
    monkeypatch.setattr("check_pwd_length.PERMIT_SHORT_PASSWORDS", True)
    check_pwd_length.check_pwd_length("petitpatapon")

Overwriting tests/concepts/mocking/monkeypatch/simple_constant_mock/test_simple_constant_mock.py


In [316]:
%%script bash --no-raise-error
pytest tests/concepts/mocking/monkeypatch/simple_constant_mock/test_simple_constant_mock.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/mocking/monkeypatch/simple_constant_mock/test_simple_constant_mock.py . [100%]

============================== 1 passed in 0.01s ===============================


## Classes de test

Les classes de test sont une manière alternative d'organiser les tests.
Les tests sont alors les méthodes de cette classe préfixées par `test`. Comme dit en début de Dojo, ces classes ne doivent pas contenir de méthode `__init__()` et doivent avoir leur nom préfixé par `Test`.

## Exemple simple :

In [320]:
%%writefile tests/concepts/test_classes/simple_example.py
#!/usr/bin/env python3

class TestClass:
    def test_method(self):
        pass

Overwriting tests/concepts/test_classes/simple_example.py


In [318]:
%%script bash --no-raise-error
pytest tests/concepts/test_classes/simple_example.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/concepts/test_classes/simple_example.py .                          [100%]

============================== 1 passed in 0.01s ===============================


## Paramétrisation de classes de test

In [324]:
%%writefile tests/concepts/test_classes/parametrization.py
#!/usr/bin/env python3
import pytest

@pytest.mark.parametrize("mybool,mystr", [(True, "foo"), (False, "bar")])
class TestClass:
    def test_mytest(self, mybool, mystr):
        print("\n%r, %r" % (mybool, mystr))

Overwriting tests/concepts/test_classes/parametrization.py


In [325]:
%%script bash --no-raise-error
pytest -v tests/concepts/test_classes/parametrization.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items

tests/concepts/test_classes/parametrization.py::TestClass::test_mytest[True-foo] PASSED [ 50%]
tests/concepts/test_classes/parametrization.py::TestClass::test_mytest[False-bar] PASSED [100%]

============================== 2 passed in 0.02s ===============================


## Fixtures dans les classes de test

On peut aussi définir une fixture dans le scope d'une classe de test.

In [326]:
%%writefile tests/concepts/test_classes/test_fixture.py
#!/usr/bin/env python3
import pytest

class TestClass:
    @pytest.fixture
    def myfixt(self):
        return 1
    
    def test_foo(self, myfixt):
        assert myfixt == 1

Overwriting tests/concepts/test_classes/test_fixture.py


In [327]:
%%script bash --no-raise-error
pytest -v tests/concepts/test_classes/test_fixture.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 1 item

tests/concepts/test_classes/test_fixture.py::TestClass::test_foo PASSED  [100%]

============================== 1 passed in 0.01s ===============================


## Intérêt des classes de test

Un intérêt majeur des classes de test se trouve dans l'utilisation de l'héritage.

In [330]:
%%writefile tests/concepts/test_classes/pros/test_example.py
#!/usr/bin/env python3
import pytest

class Bird:
    can_fly = True
    scream = "Chirp!"
    
    def get_scream(self):
        return self.scream


class Duck(Bird):
    scream = "Quack!"


class Swallow(Bird):
    pass


class Penguin(Bird):
    can_fly = False
    scream = "Je ne suis pas un pingouin."


class TestBird:
    bird_class = Bird
    bird_scream = "Chirp!"
    bird_flight_ability = True
    
    @pytest.fixture(scope="class")
    def thats_a_bird(self):
        return self.bird_class()

    def test_scream(self, thats_a_bird):
        scream = thats_a_bird.get_scream()
        expected_scream = self.bird_scream
        assert scream == expected_scream, "Scream for bird of type %r should be %r and not %r" % (thats_a_bird.__class__, expected_scream, scream)

    def test_flight_ability(self):
        flight_ability = self.bird_class.can_fly
        expected_flight_ability = self.bird_flight_ability
        assert flight_ability == expected_flight_ability


class TestDuck(TestBird):
    bird_class = Duck
    bird_scream = "Quack!"


class TestSwallow(TestBird):
    bird_class = Swallow


class TestPenguin(TestBird):
    bird_class = Penguin
    bird_flight_ability = False
    bird_scream = "Je ne suis pas un pingouin." 

Overwriting tests/concepts/test_classes/pros/test_example.py


In [332]:
%%script bash --no-raise-error
pytest -v tests/concepts/test_classes/pros/test_example.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 8 items

tests/concepts/test_classes/pros/test_example.py::TestBird::test_scream PASSED [ 12%]
tests/concepts/test_classes/pros/test_example.py::TestBird::test_flight_ability PASSED [ 25%]
tests/concepts/test_classes/pros/test_example.py::TestDuck::test_scream PASSED [ 37%]
tests/concepts/test_classes/pros/test_example.py::TestDuck::test_flight_ability PASSED [ 50%]
tests/concepts/test_classes/pros/test_example.py::TestSwallow::test_scream PASSED [ 62%]
tests/concepts/test_classes/pros/test_example.py::TestSwallow::test_flight_ability PASSED [ 75%]
tests/concepts/test_classes/pros/test_example.py::TestPenguin::test_scream PASSED [ 87%]
tests/concepts/test_classes/pros/test_example.py::TestPenguin

## Trucs :


### Capture de l'outut avec le paramètre `-s`:

In [333]:
%%writefile tests/trucs/capture_of_output/test_foo.py
def test_foo():
     print("FOO")

Overwriting tests/trucs/capture_of_output/test_foo.py


In [335]:
%%script bash --no-raise-error
pytest -s tests/trucs/capture_of_output/test_foo.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 1 item

tests/trucs/capture_of_output/test_foo.py FOO
.

============================== 1 passed in 0.01s ===============================


### Se concentrer sur les fails avec `--last-failed` et `--failed-first`:

Ces paramètres permettent respectivement de ne lancer que les tests ayant échoué et de les lancer avant ceux qui ont réussi.

In [336]:
%%writefile tests/trucs/failed_tests/test_failed.py

def test_success():
    pass

def test_failed():
    assert False

Overwriting tests/trucs/failed_tests/test_failed.py


In [337]:
%%script bash --no-raise-error
pytest -v tests/trucs/failed_tests/test_failed.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items

tests/trucs/failed_tests/test_failed.py::test_success PASSED             [ 50%]
tests/trucs/failed_tests/test_failed.py::test_failed FAILED              [100%]

=================================== FAILURES ===================================
_________________________________ test_failed __________________________________

    def test_failed():
>       assert False
E       assert False

tests/trucs/failed_tests/test_failed.py:6: AssertionError
=========================== short test summary info ============================
FAILED tests/trucs/failed_tests/test_failed.py::test_failed - assert False
========================= 1 failed, 1 passed in 0.07s ==========================


In [339]:
%%script bash --no-raise-error
pytest -v --last-failed tests/trucs/failed_tests/test_failed.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items / 1 deselected / 1 selected
run-last-failure: rerun previous 1 failure

tests/trucs/failed_tests/test_failed.py::test_failed FAILED              [100%]

=================================== FAILURES ===================================
_________________________________ test_failed __________________________________

    def test_failed():
>       assert False
E       assert False

tests/trucs/failed_tests/test_failed.py:6: AssertionError
=========================== short test summary info ============================
FAILED tests/trucs/failed_tests/test_failed.py::test_failed - assert False
======================= 1 failed, 1 deselected in 0.09s ========================


In [340]:
%%script bash --no-raise-error
pytest -v --failed-first tests/trucs/failed_tests/test_failed.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 2 items
run-last-failure: rerun previous 1 failure first

tests/trucs/failed_tests/test_failed.py::test_failed FAILED              [ 50%]
tests/trucs/failed_tests/test_failed.py::test_success PASSED             [100%]

=================================== FAILURES ===================================
_________________________________ test_failed __________________________________

    def test_failed():
>       assert False
E       assert False

tests/trucs/failed_tests/test_failed.py:6: AssertionError
=========================== short test summary info ============================
FAILED tests/trucs/failed_tests/test_failed.py::test_failed - assert False
========================= 1 failed, 1 passe

### Continuer l'exécution des tests malgré les fails :

Si je veux lancer tous les tests d'une suite de tests de 3500+ tests, et que je me doute que certains vont échouer, je peux tout-de-même faire tourner l'ensemble des tests avec `--maxfail=4000`.

In [343]:
%%writefile tests/trucs/maxfail/test_maxfail.py
#!/usr/bin/env python3

def test_fail_1():
    assert False

def test_success_1():
    pass

def test_fail_2():
    assert False

def test_success_2():
    pass

def test_fail_3():
    assert False

Overwriting tests/trucs/maxfail/test_maxfail.py


In [344]:
%%script bash --no-raise-error
pytest --maxfail=3 tests/trucs/maxfail/test_maxfail.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collected 5 items

tests/trucs/maxfail/test_maxfail.py F.F.F                                [100%]

=================================== FAILURES ===================================
_________________________________ test_fail_1 __________________________________

    def test_fail_1():
>       assert False
E       assert False

tests/trucs/maxfail/test_maxfail.py:4: AssertionError
_________________________________ test_fail_2 __________________________________

    def test_fail_2():
>       assert False
E       assert False

tests/trucs/maxfail/test_maxfail.py:10: AssertionError
_________________________________ test_fail_3 __________________________________

    def test_fail_3():
>       assert False
E       assert False

tests/trucs/maxfail/test_maxfail.py:16: AssertionError
============

### Se concentrer sur les tests frais avec `--new-first`:

In [347]:
%%writefile tests/trucs/newfirst/test_newfirst.py
#!/usr/bin/env python3

def test_1():
    pass

def test_2():
    pass

def test_3():
    pass

def test_4():
    assert False

Overwriting tests/trucs/newfirst/test_newfirst.py


In [352]:
%%script bash --no-raise-error
pytest -v --new-first tests/trucs/newfirst/test_newfirst.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0 -- /home/vmonteco/.envs/dojo-pytest/bin/python
cachedir: .pytest_cache
rootdir: /home/vmonteco/dojo-pytest
plugins: mock-3.12.0
collecting ... collected 4 items

tests/trucs/newfirst/test_newfirst.py::test_1 PASSED                     [ 25%]
tests/trucs/newfirst/test_newfirst.py::test_2 PASSED                     [ 50%]
tests/trucs/newfirst/test_newfirst.py::test_3 PASSED                     [ 75%]
tests/trucs/newfirst/test_newfirst.py::test_4 FAILED                     [100%]

=================================== FAILURES ===================================
____________________________________ test_4 ____________________________________

    def test_4():
>       assert False
E       assert False

tests/trucs/newfirst/test_newfirst.py:13: AssertionError
=========================== short test summary info ============================
FAILED tests/

### Le cache de pytest :
Les derniers résultats sont stockés dans le cache de pytest. Pytest met à disposition deux outils pour l'inspecter et le vider.

On peut ainsi l'inspecter avec `--cache-show` :

In [353]:
%%script bash --no-raise-error
cd tests/trucs/cache
pytest --tb=no
pytest --cache-show
cd ~/dojo-pytest/

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest/tests/trucs/cache
plugins: mock-3.12.0
collected 6 items

test_foo.py .F                                                           [ 33%]
subdir1/test_foo.py .F                                                   [ 66%]
subdir2/test_foo.py .F                                                   [100%]

=========================== short test summary info ============================
FAILED test_foo.py::test_fail - assert False
FAILED subdir1/test_foo.py::test_fail - assert False
FAILED subdir2/test_foo.py::test_fail - assert False
========================= 3 failed, 3 passed in 0.03s ==========================
============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest/tests/trucs/cache
plugins: mock-

Et on peut le vider avec `--cache-clear` :

In [355]:
%%script bash --no-raise-error
cd tests/trucs/cache
# --cache-clear re-runs the tests without --collect-only :)
pytest --cache-clear --collect-only
pytest --cache-show
cd ~/dojo-pytest/

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest/tests/trucs/cache
plugins: mock-3.12.0
collected 6 items

<Package cache>
  <Module test_foo.py>
    <Function test_success>
    <Function test_fail>
<Package subdir1>
  <Module test_foo.py>
    <Function test_success>
    <Function test_fail>
<Package subdir2>
  <Module test_foo.py>
    <Function test_success>
    <Function test_fail>

========================== 6 tests collected in 0.02s ==========================
============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.3.0
rootdir: /home/vmonteco/dojo-pytest/tests/trucs/cache
plugins: mock-3.12.0
cachedir: /home/vmonteco/dojo-pytest/tests/trucs/cache/.pytest_cache
----------------------------- cache values for '*' -----------------------------
cache/stepwise contains:
  []

====

# MERCI !